# Understanding Regression Through a Real-World Example

## Step 1: Import Libraries and Load Dataset

In [10]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [11]:
# Load the California Housing dataset
california = fetch_california_housing()
X = pd.DataFrame(california.data, columns=california.feature_names)
y = california.target

# Combine features and target into a single DataFrame for exploration
data = X.copy()
data['MedHouseVal'] = y

## Step 2: Exploring the Dataset

Let's explore the dataset to understand its structure and characteristics.

In [ ]:
# Display basic information
print("Dataset Info:")
print(data.info())
print("\nFirst 5 rows:")
print(data.head())

# Summary statistics
print("\nSummary Statistics:")
print(data.describe())

# Visualize feature distributions
plt.figure(figsize=(15, 10))
for i, col in enumerate(X.columns, 1):
    plt.subplot(3, 3, i)
    sns.histplot(data[col], kde=True)
    plt.title(col)
plt.tight_layout()
plt.show()

# Correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix')
plt.show()

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB
None

First 5 rows:
   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0

## Step 3: Testing Assumptions of Linear Regression

Linear regression assumes:
1. **Linearity**: The relationship between features and target is linear.
2. **Independence**: Observations are independent.
3. **Homoscedasticity**: Constant variance of residuals.
4. **Normality**: Residuals are normally distributed.
5. **No multicollinearity**: Features are not highly correlated with each other.


### Step 3.1: Linearity Check
We can check linearity using scatter plots.

In [ ]:
plt.figure(figsize=(15, 10))
for i, col in enumerate(X.columns, 1):
    plt.subplot(3, 3, i)
    plt.scatter(data[col], data['MedHouseVal'], alpha=0.3)
    plt.xlabel(col)
    plt.ylabel('MedHouseVal')
    plt.title(f'{col} vs MedHouseVal')
plt.tight_layout()
plt.show()

### Step 3.2: Multicollinearity Check
We use the Variance Inflation Factor (VIF). VIF > 5 or 10 indicates high multicollinearity.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print("\nVariance Inflation Factor (VIF):")
print(vif_data)

### Step 3.3: Normality and Homoscedasticity (Checked Post-Modeling)

# Building and Evaluating a Simple Linear Regression Model

## Step 1: Data Preprocessing
We'll split the data and scale the features.

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optional: Scale features (not strictly necessary for linear regression but can help)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Step 2: Fit the Model

In [ ]:
# Initialize and fit the model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Model Performance:")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared: {r2:.2f}")

# Display coefficients
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
})
print("\nModel Coefficients:")
print(coef_df)

## Step 3: Visualizing the Model Fit

To visualize the linear regression model, we'll plot the regression line for the `MedInc` (median income) feature against the target (`MedHouseVal`) using the test data. Since the model uses multiple features, we'll isolate `MedInc` and use the model's coefficients to approximate the relationship.


In [ ]:
# Extract MedInc from test data
med_inc_test = X_test['MedInc'].values
y_test_pred = model.predict(X_test_scaled)

# Create a simple linear regression model for MedInc alone for visualization
single_feature_model = LinearRegression()
single_feature_model.fit(X_train[['MedInc']], y_train)
y_pred_single = single_feature_model.predict(X_test[['MedInc']])

# Plot scatter and regression line
plt.figure(figsize=(8, 6))
plt.scatter(med_inc_test, y_test, alpha=0.3, label='Actual Values')
plt.plot(med_inc_test, y_pred_single, color='red', label='Regression Line')
plt.xlabel('Median Income (MedInc)')
plt.ylabel('Median House Value (MedHouseVal)')
plt.title('Linear Regression Fit for MedInc vs MedHouseVal')
plt.legend()
plt.show()

## Step 4: Residual Analysis (Normality and Homoscedasticity)

In [ ]:
# Calculate residuals
residuals = y_test - y_pred

# Residual plot
plt.figure(figsize=(12, 5))

# Scatter plot of residuals
plt.subplot(1, 2, 1)
plt.scatter(y_pred, residuals, alpha=0.3)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted')

# Histogram of residuals
plt.subplot(1, 2, 2)
sns.histplot(residuals, kde=True)
plt.title('Residuals Distribution')
plt.xlabel('Residuals')
plt.tight_layout()
plt.show()

# Q-Q plot for normality
plt.figure(figsize=(6, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot for Residuals')
plt.show()

## Step 5: Interpretation
- **RMSE**: Indicates the average error in predictions (in $100,000 units).
- **R-squared**: Proportion of variance in the target explained by the model.
- **Coefficients**: Show the impact of each feature on the target.
- **Residuals**:
  - The residual plot should show no clear pattern (homoscedasticity).
  - The histogram and Q-Q plot should indicate approximate normality.